In [ ]:
!pip install segmentation-models-pytorch

In [ ]:
# Importing Libraries
import os
import cv2
import json
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader

In [ ]:
# Defining Constants
BATCH_SIZE = 1
RADIUS = 5
THRESHOLD = 0.5
CLASSES = ["Fields"]
IMG_EXTNS = ["png", "jpg", "jpeg"]
DATA_DIR = "/content/drive/Shareddrives/Street2Sat4/S2S_Depth/valid/"
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
JSON_PATH = "/content/drive/Shareddrives/Street2Sat4/S2S_Depth_COCO/valid/_annotations.coco.json"
MODEL_NAME = "DepthExperiments512x512xception"
MODEL_DIR = "/content/drive/Shareddrives/Street2Sat4/Distance_Models/" + MODEL_NAME + "/"
OUT_DIR = "/content/drive/Shareddrives/Street2Sat4/Depth_Centr_Results/" + MODEL_NAME + "/"

In [ ]:
# Helper Functions

def to_tensor(x, **kwargs):
    return torch.from_numpy(x.transpose(2, 0, 1).astype("float32"))


def preprocess(img, **kwargs):
    img = img.astype(float)
    img = (
        255 * (img - np.min(img[:])) / (np.max(img[:]) - np.min(img[:]) + 0.1)
    ).astype(float)
    img = (img + 0.5) / 256
    gamma = -1 / np.nanmean(np.log(img))
    img = img ** (gamma)
    return img


# Post Processing
def getLargestContour(pred):
  cnts, _ = cv2.findContours((pred>0.5).astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  if(len(cnts)==0):
    return pred
  cnt  = max(cnts, key = lambda x: cv2.contourArea(x))
  new_img = np.zeros(pred.shape)
  cv2.fillPoly(new_img, pts = [cnt], color=(1.0,1.0,1.0))
  return new_img * pred


# Representative Point Extraction
def getNewMarkPos(img):
  x, y = np.where(img>THRESHOLD)
  if(len(x)==0):
    print(f"{len(x)=}")
    return np.unravel_index(np.argmax(img),img.shape)[::-1]
  x_centr, y_centr = np.mean(x), np.mean(y)
  med_val = np.median(img[x,y])
  x_new, y_new = np.where(img>=med_val)
  z = [i for i in zip(x_new,y_new)]
  return min(z, key = lambda k : (x_centr - k[0])**2 + (y_centr - k[1])**2)


def markImg(img):
  z = np.unravel_index(np.argmax(img),img.shape)[::-1]
  new_img = np.ones((*img.shape,3)) * img.reshape((*img.shape,1))
  cv2.circle(new_img, z, RADIUS, (0.0,1.0,0.0), -1)
  y1,x1 = getNewMarkPos(img)
  cv2.circle(new_img, (x1,y1), RADIUS, (0.0,0.0,1.0), -1)
  return new_img


def getZeroDist(img,pt):
  x,y = pt
  y1,x1 = np.where(img==0)
  dists = np.sqrt(((x-x1)**2 + (y-y1)**2))
  return dists.min()

def calcDists(img):
  img = np.pad(img,1)
  y0,x0 = np.unravel_index(np.argmax(img),img.shape)
  dist0 = getZeroDist(img,(x0,y0))
  y1,x1 = getNewMarkPos(img)
  dist1 = getZeroDist(img,(x1,y1))
  return [dist0,dist1]


class Data(Dataset):
    def __init__(
        self,
        images_dir,
        classes=None,
        augmentation=None,
        preprocessing=None,
        json_location=None,
        device = "cpu"
    ):
        self.ids = os.listdir(images_dir)
        self.images_fps = sorted(
            [
                os.path.join(images_dir, image_id)
                for image_id in self.ids
                if image_id.split(".")[-2].split("_")[-1] != "mask"
                and image_id.split(".") != "csv"
            ]
        )
        self.masks_fps = sorted(
            [
                os.path.join(images_dir, image_id)
                for image_id in self.ids
                if image_id.split(".")[-2].split("_")[-1] == "mask"
            ]
        )
        self.img_size = (1024, 512)
        self.preprocessing = preprocessing
        self.json_location = json_location
        self.device = DEVICE

    def __getitem__(self, i):
        image = cv2.imread(self.images_fps[i])
        mask = cv2.imread(self.masks_fps[i], cv2.IMREAD_GRAYSCALE).astype("float")
        mask = mask[0:512, 0:512]
        image = image[0:512, 0:512]
        if self.json_location:
            with open(self.json_location) as f:
                img_json = json.load(f)
                for img in img_json["images"]:
                    if img["id"] == i:
                        for j in img["extra"]["user_tags"]:
                            if j == "Right":
                                continue

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = mask.reshape((*mask.shape,1))
        """
        masks = [(mask == v) for v in self.class_values]
        mask = np.stack(masks, axis=-1).astype("float")
        """

        if self.preprocessing:
            sample = self.preprocessing(image=image, mask=mask)
            image, mask = sample["image"], sample["mask"]
        image = preprocess(image)
        mask = preprocess(mask)
        return to_tensor(image).to(self.device), to_tensor(mask).to(self.device)

    def __len__(self):
        return len(self.masks_fps)

In [ ]:
if not(os.path.isdir(OUT_DIR)):
  os.makedirs(OUT_DIR)

In [ ]:
val_dataset = Data(
    images_dir = DATA_DIR,
    classes = CLASSES,
    json_location = JSON_PATH,
    device = DEVICE,
)

val_data = DataLoader(val_dataset, batch_size = BATCH_SIZE, shuffle = False)

model = torch.load(MODEL_DIR + "/best_model.pth").to(DEVICE)

In [ ]:
dists0 = []
dists1 = []
for num,(img, mask) in enumerate(val_data):
  print(f"\rWorking on img {num+1}/{len(val_data)}", end="")
  pred = model(img)
  mask_new = mask.cpu().numpy()[0,0]
  cv2.imwrite(OUT_DIR + f"{str(num).zfill(5)}_mask.png",mask_new*255)
  img_new = img.cpu()[0].numpy().transpose(1, 2, 0)
  cv2.imwrite(OUT_DIR + f"{str(num).zfill(5)}_img.png",img_new*255)
  pred_new = pred.detach().cpu().numpy()[0,0]
  pred_new = getLargestContour(pred_new)
  # Dist estimation is only for internal analytics
  dist0,dist1 = calcDists(pred_new)
  pred_new = markImg(pred_new)
  dists0.append(dist0)
  dists1.append(dist1)
  cv2.imwrite(OUT_DIR + f"{str(num).zfill(5)}_pred.png",pred_new*255)

Working on img 55/352len(x)=0
len(x)=0
Working on img 57/352len(x)=0
len(x)=0
Working on img 136/352len(x)=0
len(x)=0
Working on img 138/352len(x)=0
len(x)=0
Working on img 139/352len(x)=0
len(x)=0
Working on img 140/352len(x)=0
len(x)=0
Working on img 158/352len(x)=0
len(x)=0
Working on img 194/352len(x)=0
len(x)=0
Working on img 330/352len(x)=0
len(x)=0
Working on img 352/352

In [ ]:
dists0 = np.array(dists0)
print(f"Method 0 (Max Pick): Mean : {np.mean(dists0)}, Median: {np.median(dists0)}")
dists1 = np.array(dists1)
print(f"Method 1 (Median Filtered Closest to Centroid Pick): Mean : {np.mean(dists1)}, Median: {np.median(dists1)}")

Method 0 (Max Pick): Mean : 4.224850513544166, Median: 3.0
Method 1 (Median Filtered Closest to Centroid Pick): Mean : 61.0033298711035, Median: 51.351572302198896


EfficientNet-b5<br>
Working on img 23/352len(x)=0<br>
Working on img 52/352len(x)=0<br>
Working on img 81/352len(x)=0<br>
Working on img 284/352len(x)=0<br>
Working on img 329/352len(x)=0<br>
Working on img 337/352len(x)=0<br>
Working on img 351/352<br>
Method 0 (Max Pick): Mean : 9.054178629356322, Median: 4.0<br>
Method 1 (Median Filtered Closest to Centroid Pick): Mean : 65.07702971240862, Median: 58.914310589820275<br>

DenseNet161<br>
Working on img 11/352len(x)=0<br>
Working on img 56/352len(x)=0<br>
Working on img 165/352len(x)=0<br>
Working on img 337/352len(x)=0<br>
Working on img 351/352<br>
Method 0 (Max Pick): Mean : 3.398718174943056, Median: 3.0<br>
Method 1 (Median Filtered Closest to Centroid Pick): Mean : 65.74666391827189, Median: 57.53506614676055<br>


Xception<br>
Working on img 55/352len(x)=0<br>
Working on img 57/352len(x)=0<br>
Working on img 136/352len(x)=0<br>
Working on img 138/352len(x)=0<br>
Working on img 139/352len(x)=0<br>
Working on img 140/352len(x)=0<br>
Working on img 158/352len(x)=0<br>
Working on img 194/352len(x)=0<br>
Working on img 330/352len(x)=0<br>
Method 0 (Max Pick): Mean : 4.224850513544166, Median: 3.0<br>
Method 1 (Median Filtered Closest to Centroid Pick): Mean : 61.0033298711035, Median: 51.351572302198896<br>

In [ ]:
!cp -r /content/drive/Shareddrives/Street2Sat4/Depth_Centr_Results /content/drive/Shareddrives/Street2Sat_Updates/